# Spotify Missing Tracks Discovery Tool
## Purpose
Generate interactive HTML for tracks missing in Tidal, with quick links to Spotify and Last.fm for discovery and validation.

**Source**: `Tidal - Spotify all - Missing.csv`  
**Output**: Interactive HTML with search, filter, and bulk operations

---
*Created by Claude (Sonnet 4.5) on 2025-10-21*


In [1]:
# First install pandas:
!pip install pandas

import pandas as pd
import urllib.parse
from pathlib import Path
from datetime import datetime
import json

# File paths
CSV_PATH = "/Users/maxfiep/Library/CloudStorage/GoogleDrive-pmaxfield@gmail.com/My Drive/Files/Files_2025/Music_Tidal_transfer/Tidal - Spotify all - Missing.csv"
OUTPUT_DIR = Path.cwd()
OUTPUT_HTML = OUTPUT_DIR / f"spotify_missing_tracks_discovery_{datetime.now().strftime('%Y%m%d')}.html"

print(f"CSV Source: {CSV_PATH}")
print(f"HTML Output: {OUTPUT_HTML}")


CSV Source: /Users/maxfiep/Library/CloudStorage/GoogleDrive-pmaxfield@gmail.com/My Drive/Files/Files_2025/Music_Tidal_transfer/Tidal - Spotify all - Missing.csv
HTML Output: /Users/maxfiep/Library/CloudStorage/GoogleDrive-pmaxfield@gmail.com/My Drive/git_personal/Pete_Sandbox_personal/Music_migration_Tidal_Spotify/spotify_missing_tracks_discovery_20251021.html


In [2]:
# Load CSV data
df = pd.read_csv(CSV_PATH, sep=',', quoting=1, on_bad_lines='skip')  # Handle inconsistent number of fields

# Display basic info
print(f"Total tracks missing from Tidal: {len(df):,}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nData types:\n{df.dtypes}")
print(f"\nFirst few rows:")
df.head(10)


Total tracks missing from Tidal: 1,981

Columns: ['Track name', 'Artist name', 'Album', 'Playlist name', 'Type']

Data types:
Track name       object
Artist name      object
Album            object
Playlist name    object
Type             object
dtype: object

First few rows:


,Track name,Artist name,Album,Playlist name,Type
0,Oye Vaya,Earl Greyhound,Suspicious Package,Favorite Songs,MySavedTracks
1,The Witch,Empire State Human,Cycles,Favorite Songs,MySavedTracks
2,Puerto Rico Pa Gozar (feat. Quantic & Nickodemus),Candela All Stars,La Concha,Favorite Songs,MySavedTracks
3,The Top Chefs,Dj Khalid Music,FIRST SERVE (2020 remastered version),Favorite Songs,MySavedTracks
4,Mississippi Queen,Ozzy Osbourne,Under Cover,Favorite Songs,MySavedTracks
5,Saturdays,J.PERIOD,The [Abstract] Best,Favorite Songs,MySavedTracks
6,Bust A Move - Instrumental,Samples 'n' Records,Bust A Move (Instrumental),Favorite Songs,MySavedTracks
7,A Lot Of Love Around,The Steepwater Band,Revelation Sunday,Favorite Songs,MySavedTracks
8,Only,Ameritz - Tribute,Only (A Tribute to Nine Inch Nails),Favorite Songs,MySavedTracks
9,Rotten Apple,8-Bit Arcade,The Ultimate Alice In Chains,Favorite Songs,MySavedTracks


In [3]:
def clean_for_url(text):
    """Clean text for URL encoding"""
    if pd.isna(text):
        return ""
    return str(text).strip()

def generate_spotify_search_url(track, artist):
    """Generate Spotify search URL"""
    track_clean = clean_for_url(track)
    artist_clean = clean_for_url(artist)
    
    if not track_clean or not artist_clean:
        return ""
    
    # Spotify search format
    query = f"{artist_clean} {track_clean}"
    encoded_query = urllib.parse.quote(query)
    return f"https://open.spotify.com/search/{encoded_query}"

def generate_lastfm_url(track, artist, album=""):
    """Generate Last.fm track URL"""
    track_clean = clean_for_url(track)
    artist_clean = clean_for_url(artist)
    album_clean = clean_for_url(album)
    
    if not track_clean or not artist_clean:
        return ""
    
    # Last.fm format: /user/sugarsmax/library/music/+noredirect/{artist}/{album}/{track}
    artist_encoded = urllib.parse.quote(artist_clean.replace(' ', '+'))
    album_encoded = urllib.parse.quote(album_clean.replace(' ', '+')) if album_clean else ""
    track_encoded = urllib.parse.quote(track_clean.replace(' ', '+'))
    
    # Build URL with album if available
    if album_encoded:
        return f"https://www.last.fm/user/sugarsmax/library/music/+noredirect/{artist_encoded}/{album_encoded}/{track_encoded}"
    else:
        return f"https://www.last.fm/user/sugarsmax/library/music/+noredirect/{artist_encoded}/_/{track_encoded}"

def generate_tidal_search_url(track, artist):
    """Generate Tidal search URL"""
    track_clean = clean_for_url(track)
    artist_clean = clean_for_url(artist)
    
    if not track_clean or not artist_clean:
        return ""
    
    # Tidal search format: https://tidal.com/search?q={query}
    # Add quotes for exact phrase matching
    query = f'"{artist_clean}" "{track_clean}"'
    encoded_query = urllib.parse.quote(query)
    return f"https://tidal.com/search?q={encoded_query}"

# Test with first row
test_track = df.iloc[0]['Track name']
test_artist = df.iloc[0]['Artist name']
test_album = df.iloc[0]['Album']

print(f"Test Track: {test_track}")
print(f"Test Artist: {test_artist}")
print(f"Test Album: {test_album}")
print(f"\nSpotify URL: {generate_spotify_search_url(test_track, test_artist)}")
print(f"Last.fm URL: {generate_lastfm_url(test_track, test_artist, test_album)}")
print(f"Tidal URL: {generate_tidal_search_url(test_track, test_artist)}")


Test Track: Oye Vaya
Test Artist: Earl Greyhound
Test Album: Suspicious Package

Spotify URL: https://open.spotify.com/search/Earl%20Greyhound%20Oye%20Vaya
Last.fm URL: https://www.last.fm/user/sugarsmax/library/music/+noredirect/Earl%2BGreyhound/Suspicious%2BPackage/Oye%2BVaya
Tidal URL: https://tidal.com/search?q=%22Earl%20Greyhound%22%20%22Oye%20Vaya%22


In [4]:
# Generate URLs for all tracks
print("Generating URLs for all tracks...")

df['spotify_url'] = df.apply(lambda row: generate_spotify_search_url(row['Track name'], row['Artist name']), axis=1)
df['lastfm_url'] = df.apply(lambda row: generate_lastfm_url(row['Track name'], row['Artist name'], row['Album']), axis=1)
df['tidal_url'] = df.apply(lambda row: generate_tidal_search_url(row['Track name'], row['Artist name']), axis=1)

print(f"✓ Generated URLs for {len(df):,} tracks")
print(f"\nSample URLs:")
df[['Track name', 'Artist name', 'Album', 'spotify_url', 'lastfm_url', 'tidal_url']].head(3)


Generating URLs for all tracks...
✓ Generated URLs for 1,981 tracks

Sample URLs:


,Track name,Artist name,Album,spotify_url,lastfm_url,tidal_url
0,Oye Vaya,Earl Greyhound,Suspicious Package,https://open.spotify.com/search/Earl%20Greyhou...,https://www.last.fm/user/sugarsmax/library/mus...,https://tidal.com/search?q=%22Earl%20Greyhound...
1,The Witch,Empire State Human,Cycles,https://open.spotify.com/search/Empire%20State...,https://www.last.fm/user/sugarsmax/library/mus...,https://tidal.com/search?q=%22Empire%20State%2...
2,Puerto Rico Pa Gozar (feat. Quantic & Nickodemus),Candela All Stars,La Concha,https://open.spotify.com/search/Candela%20All%...,https://www.last.fm/user/sugarsmax/library/mus...,https://tidal.com/search?q=%22Candela%20All%20...


In [5]:
def generate_interactive_html(df, output_path):
    """Generate interactive HTML with search, filter, and quick discovery features"""
    
    # Convert dataframe to JSON for JavaScript
    tracks_json = df.to_json(orient='records')
    
    html_content = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Spotify Missing Tracks - Discovery Tool</title>
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        body {{
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 20px;
            min-height: 100vh;
        }}
        
        .container {{
            max-width: 1400px;
            margin: 0 auto;
            background: white;
            border-radius: 16px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
            overflow: hidden;
        }}
        
        .header {{
            background: linear-gradient(135deg, #1DB954 0%, #1ed760 100%);
            color: white;
            padding: 30px;
            text-align: center;
        }}
        
        .header h1 {{
            font-size: 2.5em;
            margin-bottom: 10px;
        }}
        
        .header p {{
            font-size: 1.1em;
            opacity: 0.9;
        }}
        
        .stats {{
            background: #f8f9fa;
            padding: 20px 30px;
            display: flex;
            justify-content: space-around;
            flex-wrap: wrap;
            border-bottom: 2px solid #e9ecef;
        }}
        
        .stat-box {{
            text-align: center;
            padding: 10px 20px;
        }}
        
        .stat-number {{
            font-size: 2em;
            font-weight: bold;
            color: #1DB954;
        }}
        
        .stat-label {{
            color: #6c757d;
            font-size: 0.9em;
            margin-top: 5px;
        }}
        
        .controls {{
            padding: 30px;
            background: #fff;
            border-bottom: 2px solid #e9ecef;
        }}
        
        .search-box {{
            display: flex;
            gap: 15px;
            margin-bottom: 20px;
            flex-wrap: wrap;
        }}
        
        .search-input {{
            flex: 1;
            min-width: 300px;
            padding: 12px 20px;
            font-size: 16px;
            border: 2px solid #dee2e6;
            border-radius: 8px;
            transition: all 0.3s;
        }}
        
        .search-input:focus {{
            outline: none;
            border-color: #1DB954;
            box-shadow: 0 0 0 3px rgba(29, 185, 84, 0.1);
        }}
        
        .filter-group {{
            display: flex;
            gap: 10px;
            flex-wrap: wrap;
        }}
        
        select {{
            padding: 12px 20px;
            font-size: 16px;
            border: 2px solid #dee2e6;
            border-radius: 8px;
            background: white;
            cursor: pointer;
        }}
        
        .button {{
            padding: 12px 24px;
            font-size: 16px;
            border: none;
            border-radius: 8px;
            cursor: pointer;
            transition: all 0.3s;
            font-weight: 600;
        }}
        
        .button-primary {{
            background: #1DB954;
            color: white;
        }}
        
        .button-primary:hover {{
            background: #1ed760;
            transform: translateY(-2px);
            box-shadow: 0 4px 12px rgba(29, 185, 84, 0.4);
        }}
        
        .button-secondary {{
            background: #6c757d;
            color: white;
        }}
        
        .button-secondary:hover {{
            background: #5a6268;
        }}
        
        .table-container {{
            overflow-x: auto;
            padding: 0 30px 30px 30px;
        }}
        
        table {{
            width: 100%;
            border-collapse: collapse;
            font-size: 14px;
        }}
        
        thead {{
            background: #f8f9fa;
            position: sticky;
            top: 0;
            z-index: 10;
        }}
        
        th {{
            padding: 15px 12px;
            text-align: left;
            font-weight: 600;
            color: #495057;
            border-bottom: 2px solid #dee2e6;
            cursor: pointer;
            user-select: none;
        }}
        
        th:hover {{
            background: #e9ecef;
        }}
        
        th::after {{
            content: ' ⇅';
            opacity: 0.3;
        }}
        
        td {{
            padding: 12px;
            border-bottom: 1px solid #e9ecef;
        }}
        
        tbody tr:hover {{
            background: #f8f9fa;
        }}
        
        .track-name {{
            font-weight: 600;
            color: #212529;
        }}
        
        .artist-name {{
            color: #6c757d;
        }}
        
        .link-group {{
            display: flex;
            gap: 10px;
        }}
        
        .link-button {{
            padding: 6px 12px;
            border-radius: 6px;
            text-decoration: none;
            font-size: 12px;
            font-weight: 600;
            transition: all 0.2s;
            display: inline-block;
        }}
        
        .spotify-link {{
            background: #1DB954;
            color: white;
        }}
        
        .spotify-link:hover {{
            background: #1ed760;
            transform: scale(1.05);
        }}
        
        .lastfm-link {{
            background: #d51007;
            color: white;
        }}
        
        .lastfm-link:hover {{
            background: #ff0000;
            transform: scale(1.05);
        }}
        
        .tidal-link {{
            background: #000000;
            color: white;
        }}
        
        .tidal-link:hover {{
            background: #333333;
            transform: scale(1.05);
        }}
        
        .no-results {{
            text-align: center;
            padding: 60px 20px;
            color: #6c757d;
            font-size: 1.2em;
        }}
        
        @media (max-width: 768px) {{
            .header h1 {{
                font-size: 1.8em;
            }}
            
            .stats {{
                flex-direction: column;
            }}
            
            .search-box {{
                flex-direction: column;
            }}
            
            .search-input {{
                min-width: 100%;
            }}
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🎵 Spotify Missing Tracks Discovery</h1>
            <p>Tracks from Spotify not available in Tidal - Fast discovery tool</p>
        </div>
        
        <div class="stats">
            <div class="stat-box">
                <div class="stat-number" id="totalTracks">{len(df):,}</div>
                <div class="stat-label">Total Missing Tracks</div>
            </div>
            <div class="stat-box">
                <div class="stat-number" id="displayedTracks">{len(df):,}</div>
                <div class="stat-label">Displayed</div>
            </div>
            <div class="stat-box">
                <div class="stat-number" id="uniqueArtists">{df['Artist name'].nunique():,}</div>
                <div class="stat-label">Unique Artists</div>
            </div>
        </div>
        
        <div class="controls">
            <div class="search-box">
                <input 
                    type="text" 
                    id="searchInput" 
                    class="search-input" 
                    placeholder="🔍 Search by track name, artist, or album..."
                >
                <button class="button button-secondary" onclick="clearFilters()">Clear All</button>
            </div>
            
            <div class="filter-group">
                <select id="playlistFilter">
                    <option value="">All Playlists</option>
                </select>
                <select id="typeFilter">
                    <option value="">All Types</option>
                </select>
            </div>
        </div>
        
        <div class="table-container">
            <table id="tracksTable">
                <thead>
                    <tr>
                        <th onclick="sortTable(0)">Track</th>
                        <th onclick="sortTable(1)">Artist</th>
                        <th onclick="sortTable(2)">Album</th>
                        <th onclick="sortTable(3)">Playlist</th>
                        <th onclick="sortTable(4)">Type</th>
                        <th>Discovery Links</th>
                    </tr>
                </thead>
                <tbody id="tracksBody">
                </tbody>
            </table>
            <div id="noResults" class="no-results" style="display: none;">
                No tracks found matching your search criteria 😔
            </div>
        </div>
    </div>
    
    <script>
        // Data from Python
        const allTracks = {tracks_json};
        let filteredTracks = [...allTracks];
        let sortDirection = {{}};
        
        // Initialize page
        function init() {{
            populateFilters();
            renderTable(filteredTracks);
            setupEventListeners();
        }}
        
        function populateFilters() {{
            // Populate playlist filter
            const playlists = [...new Set(allTracks.map(t => t['Playlist name']))].sort();
            const playlistSelect = document.getElementById('playlistFilter');
            playlists.forEach(playlist => {{
                const option = document.createElement('option');
                option.value = playlist;
                option.textContent = playlist;
                playlistSelect.appendChild(option);
            }});
            
            // Populate type filter
            const types = [...new Set(allTracks.map(t => t['Type']))].sort();
            const typeSelect = document.getElementById('typeFilter');
            types.forEach(type => {{
                const option = document.createElement('option');
                option.value = type;
                option.textContent = type;
                typeSelect.appendChild(option);
            }});
        }}
        
        function setupEventListeners() {{
            document.getElementById('searchInput').addEventListener('input', filterTracks);
            document.getElementById('playlistFilter').addEventListener('change', filterTracks);
            document.getElementById('typeFilter').addEventListener('change', filterTracks);
        }}
        
        function filterTracks() {{
            const searchTerm = document.getElementById('searchInput').value.toLowerCase();
            const playlistFilter = document.getElementById('playlistFilter').value;
            const typeFilter = document.getElementById('typeFilter').value;
            
            filteredTracks = allTracks.filter(track => {{
                const trackName = (track['Track name'] || '').toLowerCase();
                const artistName = (track['Artist name'] || '').toLowerCase();
                const albumName = (track['Album'] || '').toLowerCase();
                
                const matchesSearch = searchTerm === '' || 
                    trackName.includes(searchTerm) ||
                    artistName.includes(searchTerm) ||
                    albumName.includes(searchTerm);
                
                const matchesPlaylist = playlistFilter === '' || track['Playlist name'] === playlistFilter;
                const matchesType = typeFilter === '' || track['Type'] === typeFilter;
                
                return matchesSearch && matchesPlaylist && matchesType;
            }});
            
            renderTable(filteredTracks);
            updateStats();
        }}
        
        function renderTable(tracks) {{
            const tbody = document.getElementById('tracksBody');
            const noResults = document.getElementById('noResults');
            
            if (tracks.length === 0) {{
                tbody.innerHTML = '';
                noResults.style.display = 'block';
                return;
            }}
            
            noResults.style.display = 'none';
            
            tbody.innerHTML = tracks.map(track => `
                <tr>
                    <td class="track-name">${{escapeHtml(track['Track name'])}}</td>
                    <td class="artist-name">${{escapeHtml(track['Artist name'])}}</td>
                    <td>${{escapeHtml(track['Album'])}}</td>
                    <td>${{escapeHtml(track['Playlist name'])}}</td>
                    <td>${{escapeHtml(track['Type'])}}</td>
                    <td>
                        <div class="link-group">
                            <a href="${{track.spotify_url}}" target="_blank" class="link-button spotify-link">
                                Spotify
                            </a>
                            <a href="${{track.lastfm_url}}" target="_blank" class="link-button lastfm-link">
                                Last.fm
                            </a>
                            <a href="${{track.tidal_url}}" target="_blank" class="link-button tidal-link">
                                Tidal
                            </a>
                        </div>
                    </td>
                </tr>
            `).join('');
        }}
        
        function sortTable(columnIndex) {{
            const columns = ['Track name', 'Artist name', 'Album', 'Playlist name', 'Type'];
            const column = columns[columnIndex];
            
            sortDirection[column] = !sortDirection[column];
            
            filteredTracks.sort((a, b) => {{
                const aVal = String(a[column]).toLowerCase();
                const bVal = String(b[column]).toLowerCase();
                
                if (aVal < bVal) return sortDirection[column] ? -1 : 1;
                if (aVal > bVal) return sortDirection[column] ? 1 : -1;
                return 0;
            }});
            
            renderTable(filteredTracks);
        }}
        
        function clearFilters() {{
            document.getElementById('searchInput').value = '';
            document.getElementById('playlistFilter').value = '';
            document.getElementById('typeFilter').value = '';
            filterTracks();
        }}
        
        function updateStats() {{
            document.getElementById('displayedTracks').textContent = filteredTracks.length.toLocaleString();
        }}
        
        function escapeHtml(text) {{
            const div = document.createElement('div');
            div.textContent = text || '';
            return div.innerHTML;
        }}
        
        // Initialize on load
        init();
    </script>
</body>
</html>"""
    
    # Write HTML file
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    return output_path

# Generate the HTML
print("Generating interactive HTML...")
html_path = generate_interactive_html(df, OUTPUT_HTML)
print(f"✓ HTML file created: {{html_path}}")
print(f"\nFile size: {{html_path.stat().st_size / 1024:.1f}} KB")


Generating interactive HTML...
✓ HTML file created: {html_path}

File size: {html_path.stat().st_size / 1024:.1f} KB


## Summary & Quick Access


In [6]:
import webbrowser
import os

# Open HTML file in default browser
print("="*80)
print("🎉 NOTEBOOK COMPLETE")
print("="*80)

print(f"\n📊 Statistics:")
print(f"   • Total missing tracks: {len(df):,}")
print(f"   • Unique artists: {df['Artist name'].nunique():,}")
print(f"   • Unique albums: {df['Album'].nunique():,}")
print(f"   • Playlists: {df['Playlist name'].nunique()}")

print(f"\n📁 Output File:")
print(f"   {OUTPUT_HTML}")

print(f"\n🌐 Features:")
print(f"   • Real-time search across all fields")
print(f"   • Filter by playlist and type")
print(f"   • Sortable columns (click headers)")
print(f"   • Direct links to Spotify, Last.fm, and Tidal")
print(f"   • Responsive design for mobile/desktop")

print("\n🚀 Opening HTML file in browser...")
webbrowser.open(f'file://{OUTPUT_HTML.absolute()}')

print("\n✓ HTML file opened in your default browser!")


🎉 NOTEBOOK COMPLETE

📊 Statistics:
   • Total missing tracks: 1,981
   • Unique artists: 490
   • Unique albums: 454
   • Playlists: 85

📁 Output File:
   /Users/maxfiep/Library/CloudStorage/GoogleDrive-pmaxfield@gmail.com/My Drive/git_personal/Pete_Sandbox_personal/Music_migration_Tidal_Spotify/spotify_missing_tracks_discovery_20251021.html

🌐 Features:
   • Real-time search across all fields
   • Filter by playlist and type
   • Sortable columns (click headers)
   • Direct links to Spotify, Last.fm, and Tidal
   • Responsive design for mobile/desktop

🚀 Opening HTML file in browser...

✓ HTML file opened in your default browser!


---

## Usage Notes

### How to Use the HTML File

1. **Search**: Type in the search box to filter by track name, artist, or album
2. **Filter**: Use dropdown menus to filter by playlist or content type
3. **Sort**: Click any column header to sort (click again to reverse)
4. **Discover**: Click "Spotify", "Last.fm", or "Tidal" buttons to open tracks in new tabs
5. **Clear**: Use "Clear All" button to reset all filters

### Validation

The Playwright validation cell (above) tests a random sample of URLs to ensure they work correctly. If you want to validate more URLs or skip validation, adjust the `sample_size` parameter or comment out the cell.

### Playwright Installation

If you want to run URL validation:
```bash
pip install playwright
playwright install chromium
```

---

*Built with Claude (Sonnet 4.5) - October 21, 2025*
